<u><h2>Why MediaPipe Hands?</h2></u>

<img src="images/hand-keypoints.png" />

<h2><u>Final Output</u></h2>

<img src="images/output.png" />

In [1]:
import cv2 # pip install opencv-python
import time
import random
import numpy as np # pip install numpy
import mediapipe as mp # pip install mediapipe
from google.protobuf.json_format import MessageToDict # pip install protobuf

In [2]:
mp_hands = mp.solutions.hands

hand_detector = mp_hands.Hands(
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

In [26]:
def generate_random_ball_cordinates():
    num_balls = random.randrange(0, 5)
    ball_init_cordinates = []
    ball_colors = []

    while len(ball_init_cordinates) != num_balls:
        ball_x = random.randrange(ball_radius, frame_length - 100)
        ball_cordinate = [ball_x, window_height - random.randrange(-50, 0)]
        
        while ball_cordinate in ball_init_cordinates:
            ball_x = random.randrange(ball_radius, frame_length - 100)
            ball_cordinate = [ball_x, window_height - random.randrange(-50, 0)]
        
        tl1 =  ball_cordinate[0] - ball_radius, ball_cordinate[1] - ball_radius
        tr1 = ball_cordinate[0] + ball_radius, ball_cordinate[1] - ball_radius
        bl1 = ball_cordinate[0] - ball_radius, ball_cordinate[1] + ball_radius
        overlapped = False

        for center in ball_init_cordinates:
            tl2 = center[0] - ball_radius, center[1] - ball_radius
            tr2 = center[0] + ball_radius, center[1] - ball_radius
            bl2 = center[0] - ball_radius, center[1] + ball_radius
            
            if (tl1[0] <= tl2[0] <= tr1[0] or tl1[0] <= tr2[0] <= tr1[0]) and (tl1[1] <= tl2[1] <= bl1[1] or tl1[1] <= bl2[1] <= bl1[1]):
                overlapped = True
                break
            
        if not overlapped:
            ball_init_cordinates.append(ball_cordinate)
            ball_colors.append(random.choice(rand_colors))
    return ball_init_cordinates, ball_colors

In [13]:
window_height = 650
window_length = 950
frame_length = 750
ball_radius = 30
ball_move_step = 5
ball_move_delay = 2
boundary_y = 550
fps, pre_frame_time, new_frame_time = 0, 0, 0
counter = 0
catched = 0
missed = 0
rand_colors = [(0, 255, 255), (60, 20, 220), (0, 69, 255), (139, 139, 0), (255, 144, 30), (130, 0, 75)]
ball_init_cordinates, ball_colors = generate_random_ball_cordinates()

source = cv2.VideoCapture(0)

while source.isOpened():
    ret, frame = source.read()

    if ret:
        new_frame_time = time.time()
        fps = int(1 / (new_frame_time - pre_frame_time))
        pre_frame_time = new_frame_time

        panel = np.zeros((window_height, window_length, 3), dtype="uint8") 
        image = cv2.resize(frame, (frame_length, window_height))
        image = cv2.flip(image, 1)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hand_detector.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        index_fingers_x = None
        index_fingers_y = None
        
        if results.multi_hand_landmarks:
            for landmark in results.multi_hand_landmarks:
                cordinate_data = MessageToDict(landmark)["landmark"]
                index_finger_tip_cordinate = cordinate_data[8]
                index_finger_x = int(index_finger_tip_cordinate["x"] * frame_length)
                index_finger_y = int(index_finger_tip_cordinate["y"] * window_height)
                
                cv2.line(image, (index_finger_x, index_finger_y - 40), (index_finger_x, index_finger_y - 20), (0, 0, 255), 3)
                cv2.line(image, (index_finger_x, index_finger_y + 40), (index_finger_x, index_finger_y + 20), (0, 0, 255), 3)
                cv2.line(image, (index_finger_x - 40, index_finger_y), (index_finger_x - 20, index_finger_y), (0, 0, 255), 3)
                cv2.line(image, (index_finger_x + 40, index_finger_y), (index_finger_x + 20, index_finger_y), (0, 0, 255), 3)
                cv2.circle(image, (index_finger_x, index_finger_y), 5, (0, 255, 0), -1)
                cv2.circle(image, (index_finger_x, index_finger_y), 8, (255, 255, 255), 1)
                
                if index_fingers_x == None:
                    index_fingers_x = []
                    index_fingers_y = []
                    
                index_fingers_x.append(index_finger_x)
                index_fingers_y.append(index_finger_y)
                
        if counter % ball_move_delay == 0:
            for c in ball_init_cordinates:
                c[1] -= ball_move_step
        
        released_balls = []
        boundary_passed_balls = 0
        
        for i in range(len(ball_init_cordinates)):
            c = ball_init_cordinates[i]
            
            if c[1] <= 0:
                released_balls.append(i)
                missed += 1
            elif c[1] <= boundary_y:
                boundary_passed_balls += 1
            
        for b in range(len(released_balls)):
            ball_init_cordinates.pop(b)
            ball_colors.pop(b)
        
        overlapped = []
        if index_fingers_x and index_fingers_y:
            for i in range(len(ball_init_cordinates)):
                c = ball_init_cordinates[i]

                tl1 =  c[0] - ball_radius, c[1] - ball_radius
                tr1 = c[0] + ball_radius, c[1] - ball_radius
                bl1 = c[0] - ball_radius, c[1] + ball_radius
                
                for j in range(len(index_fingers_x)):
                    index_finger_x = index_fingers_x[j]
                    index_finger_y = index_fingers_y[j]
                               
                    tl2 =  index_finger_x - 5, index_finger_y - 5
                    tr2 = index_finger_x + 5, index_finger_y - 5
                    bl2 = index_finger_x - 5, index_finger_y + 5

                    if (tl1[0] <= tl2[0] <= tr1[0] and tl1[0] <= tr2[0] <= tr1[0]) and (tl1[1] <= tl2[1] <= bl1[1] and tl1[1] <= bl2[1] <= bl1[1]):
                        overlapped.append(i)
                        cv2.circle(image, (c[0], c[1]), ball_radius, (0, 0, 255), -1)
                    
        catched += len(overlapped)
        for i in overlapped:
            ball_init_cordinates.pop(i)
            ball_colors.pop(i)            
        
        if boundary_passed_balls == len(ball_init_cordinates):
            cordinates, colors = generate_random_ball_cordinates()
            ball_init_cordinates.extend(cordinates)
            ball_colors.extend(colors)
        
        for i in range(len(ball_init_cordinates)):
            c = ball_init_cordinates[i]
            cv2.circle(image, c, ball_radius, ball_colors[i], -1)
            cv2.circle(image, c, ball_radius+1, (255, 255, 255), 2)     
            
        panel[:window_height, :frame_length, :] = image
        
        missed_formatted = missed
        catched_formatted = catched
        
        if missed < 10:
            missed_formatted = "00" + str(missed)
        elif missed <= 99:
            missed_formatted = "0" + str(missed)
        elif missed >= 1000:
            missed_formatted = "{0:.1f}K".format(missed / 1000) # 1.5k
        else:
            missed_formatted = str(missed)

        if catched < 10:
            catched_formatted = "00" + str(catched)
        elif catched <= 99:
            catched_formatted = "0" + str(catched)
        elif catched >= 1000:
            catched_formatted = "{0:.1f}K".format(catched / 1000)
        else:
            catched_formatted = str(catched)

        cv2.circle(panel, (850, 120), 80, (255, 0, 0), 3)
        cv2.circle(panel, (850, 320), 80, (0, 255, 0), 3)
        cv2.circle(panel, (850, 520), 80, (0, 0, 255), 3)
        cv2.putText(panel, "FPS", (825, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)
        cv2.putText(panel, "CATCHED", (800, 370), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        cv2.putText(panel, "MISSED", (810, 570), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        cv2.putText(panel, "{:3}".format(fps), (796, 120), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 2)
        cv2.putText(panel, catched_formatted, (804, 320), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
        cv2.putText(panel, missed_formatted, (805, 520), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)
        
        cv2.imshow("LIVE | Ball Catcher", panel)
        k = cv2.waitKey(1)

        if k == 27:
            break
        counter += 1

source.release()
cv2.destroyAllWindows()

<u><h4>References</h4></u>
<br>
<a href='https://google.github.io/mediapipe/solutions/hands.html'>MediaPipe-Hands</a>